# Pretrain

In [7]:
from forgebox.imports import *
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [4]:
data = pd.concat(list(map(pd.read_csv,["train.csv", ])))

Create a dataset based on purely text data

In [5]:
data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n',''))

text  = '\n'.join(data.excerpt.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [8]:
model_name = 'roberta-large'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./rbt_lg_pre');

Pretrain use Masked Language Modeling

In [9]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./weights/pre_rbt_lg", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=18,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

/anaconda3/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [10]:
trainer.train()

***** Running training *****
  Num examples = 2834
  Num Epochs = 5
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 595
/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
200,No log,1.156952
400,No log,1.070096


***** Running Evaluation *****
  Num examples = 2834
  Batch size = 36
Saving model checkpoint to ./weights/pre_rbt_lg/checkpoint-200
Configuration saved in ./weights/pre_rbt_lg/checkpoint-200/config.json
Model weights saved in ./weights/pre_rbt_lg/checkpoint-200/pytorch_model.bin
/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2834
  Batch size = 36
Saving model checkpoint to ./weights/pre_rbt_lg/checkpoint-400
Configuration saved in ./weights/pre_rbt_lg/checkpoint-400/config.json
Model weights saved in ./weights/pre_rbt_lg/checkpoint-400/pytorch_model.bin
/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; 

TrainOutput(global_step=595, training_loss=1.215082479525013, metrics={'train_runtime': 739.4768, 'train_samples_per_second': 19.162, 'train_steps_per_second': 0.805, 'total_flos': 7669982690922552.0, 'train_loss': 1.215082479525013, 'epoch': 5.0})

In [11]:
trainer.save_model("weights/pre_rbtlg")

Saving model checkpoint to weights/pre_rbtlg
Configuration saved in weights/pre_rbtlg/config.json
Model weights saved in weights/pre_rbtlg/pytorch_model.bin
